# Create the document to be reviewed

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('../data/experimental_data.csv')
data = data.reset_index()
completions = data[(data['PennElementName'] == 'BlankAnswer') & (data['Parameter'] == 'Final')]
completions = completions.rename(columns={'index':'ID','Value':'Original_Value'})
prepreprocessed = completions[['ID',"MD5 hash of participant's IP address",'Item','Condition','Original_Value']]
prepreprocessed['Changed_Value'] = ''
prepreprocessed['IsQuestionable'] = False
prepreprocessed['Original_Value'] = prepreprocessed['Original_Value'].apply(lambda x: x.strip('%0A'))

prepreprocessed = prepreprocessed.set_index(['Item','Condition'])
prepreprocessed = prepreprocessed.sort_index()

prepreprocessed

/var/folders/6g/v4dn3hc95x1cy5j6qz2c4t6r0000gn/T/ipykernel_45300/3095525927.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prepreprocessed['Changed_Value'] = ''
/var/folders/6g/v4dn3hc95x1cy5j6qz2c4t6r0000gn/T/ipykernel_45300/3095525927.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prepreprocessed['IsQuestionable'] = False
/var/folders/6g/v4dn3hc95x1cy5j6qz2c4t6r0000gn/T/ipykernel_45300/3095525927.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

ID MD5 hash of participant's IP address  \
Item Condition                                               
1    A           1228     2817aa3ba5ce3496c4222adb7a6aa37c   
     A           6957     83d8403f227baca0464166c404dc0dc2   
     A          11881     14399e53229004f2c85ec3810d5cf776   
     B           2511     9a8edf1b84fcdd4e9692881d31a5a26e   
     B           4146     559a558cc071b904f807f21b428370da   
...               ...                                  ...   
80   D           5354     38515a2d399231bb4340622c162d7b1a   
     D           6172     bac07a9a07d31bd971ed4288f10499d6   
     D          11089     4f1511992b86c3d28f958bcbf4d56511   
     D          13549     773048bc5ea832a6a6fb93f6da578bc6   
     D          15186     bc3b5c51f98f643be9a99134572a2170   

                                                   Original_Value  \
Item Condition                                                      
1    A                                  think about what is next.   
     A                                               take a break   
     A                               Replace the broken apparatus   
     B                                           do more science.   
     B                                     pause work for the day   
...                                                           ...   
80   D                                                   accident   
     D                                                     damage   
     D          the paramedics in the elevator%2C as there wer...   
     D                                  be rushed to the hospital   
     D                                              care for them   

               Changed_Value  IsQuestionable  
Item Condition                                
1    A                                 False  
     A                                 False  
     A                                 False  
     B                                 False  
     B                                 False  
...                      ...             ...  
80   D                                 False  
     D                                 False  
     D                                 False  
     D                                 False  
     D                                 False  

[1600 rows x 5 columns]

In [4]:
prepreprocessed.to_excel('StimuliReview.xlsx')

# Merge the changes with the original stimuli

In [48]:
# get the changes
complete_reviews = pd.read_csv('StimuliReviewCompleted.csv',sep=';')
changes = complete_reviews[['ID','Item','Condition','Discussion_TL']]
changes = changes.rename(columns={'Discussion_TL':'Changes'})
changes['Item']=changes['Item'].fillna(method='ffill').apply(lambda x: int(x)) # necessary to mark Item in every column
changes.head()

,ID,Item,Condition,Changes
0,1228,1,A,think
1,6957,1,A,pause
2,11881,1,A,replace
3,2511,1,B,NaN
4,4146,1,B,pause


In [62]:
# merge the changes into the original
merged = pd.merge(prepreprocessed.set_index('ID'),
                   changes.set_index('ID'),
                   how='outer',left_index=True, right_index=True) # merge by ID

# the final Preprocessed Values are the values of Changes for rows where they exist; if they don't, then it's the original value
merged['Preprocessed value'] = merged['Changes'].fillna(merged['Original_Value'])

# some final cleaning
merged['Preprocessed value'] = merged['Preprocessed value'].apply(lambda x: x.replace('%2C',',').strip('.').lower())
preprocessed = merged[["MD5 hash of participant's IP address",'Item','Condition','Preprocessed value']]
preprocessed

,MD5 hash of participant's IP address,Item,Condition,Preprocessed value
ID,,,,
1228,2817aa3ba5ce3496c4222adb7a6aa37c,1,A,think
6957,83d8403f227baca0464166c404dc0dc2,1,A,pause
11881,14399e53229004f2c85ec3810d5cf776,1,A,replace
2511,9a8edf1b84fcdd4e9692881d31a5a26e,1,B,do more science
4146,559a558cc071b904f807f21b428370da,1,B,pause
...,...,...,...,...
5354,38515a2d399231bb4340622c162d7b1a,80,D,accident
6172,bac07a9a07d31bd971ed4288f10499d6,80,D,damage
11089,4f1511992b86c3d28f958bcbf4d56511,80,D,"the paramedics in the elevator, as there were ..."


In [63]:
preprocessed.to_csv('PreprocessedData.csv') # export to CSV

# Find participants with high rates of ungrammatical answers

In [66]:
questionable_rate = complete_reviews.groupby("MD5 hash of participant's IP address").mean('IsQuestionable')
questionable_rate.to_csv('questionable_rate.csv')